In [2]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from sklearn.cluster import KMeans
from scipy.ndimage.morphology import binary_fill_holes
import cv2 
import seaborn as sns              # For pairplots
import matplotlib.cm as cm         # Color map
%matplotlib inline 
import random
import warnings
from itertools import chain

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras import backend as K

import tensorflow as tf

Using TensorFlow backend.


In [3]:
TEST_DIR = 'C:/input/stage2_test_final'

In [4]:
IMG_DIR_NAME = 'images'   # Folder name including the image
MASK_DIR_NAME = 'masks'   # Folder name including the masks
print('TEST_DIR = {}'.format(TEST_DIR))

TEST_DIR = C:/input/stage2_test_final


In [6]:
test_df = pd.read_csv('./test_df.csv')
test_ids = test_df['img_id']

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1      

from tqdm import tqdm
# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_DIR +'/' + id_
    img = cv2.imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    if (test_df['HSV_CLUSTER'].iloc[n]==1): 
        img = cv2.bitwise_not(img)
    sizes_test.append([img.shape[0], img.shape[1]])
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=-1)
    X_test[n] = img

print('Done!')

Getting and resizing test images ... 


100%|██████████████████████████████████████████████████████████████████████████████| 3019/3019 [00:35<00:00, 84.09it/s]


Done!


In [9]:
# Custom loss function
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

# Build U-Net model
inputs = Input((256, 256, 1))
inp_norm = BatchNormalization(axis=1)(inputs)
s = Lambda(lambda x: x / 255) (inp_norm)


c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
c1 = BatchNormalization(axis=1)(c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
c2 = BatchNormalization(axis=1)(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
c3 = BatchNormalization(axis=1)(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
c4 = BatchNormalization(axis=1)(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)
c5 = BatchNormalization(axis=1)(c5)

u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)
c6 = BatchNormalization(axis=1)(c6)

u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)
c7 = BatchNormalization(axis=1)(c7)

u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)
c8 = BatchNormalization(axis=1)(c8)

u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)
c9 = BatchNormalization(axis=1)(c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss = bce_dice_loss, metrics=[dice_coef])
model.summary()

model = load_model('pre+augumention.h5', custom_objects={'bce_dice_loss': bce_dice_loss, 'dice_coef': dice_coef})

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 256, 256, 1)  1024        input_3[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 256, 256, 1)  0           batch_normalization_21[0][0]     
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 256, 256, 32) 320         lambda_3[0][0]                   
__________________________________________________________________________________________________
dropout_19

In [33]:
preds_test_upsampled = []
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))
    
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('s2-U-net+Aug.csv', index=False)


3019/3019 [==============================] - ETA: 2:27:0 - ETA: 2:27:1 - ETA: 2:30:4 - ETA: 2:33:1 - ETA: 2:30:5 - ETA: 2:29:3 - ETA: 2:28:0 - ETA: 2:26:1 - ETA: 2:23:4 - ETA: 2:21:1 - ETA: 2:18:4 - ETA: 2:16:4 - ETA: 2:14:2 - ETA: 2:11:5 - ETA: 2:09:4 - ETA: 2:07:2 - ETA: 2:05:2 - ETA: 2:03:1 - ETA: 2:01:2 - ETA: 1:59:2 - ETA: 1:57:3 - ETA: 1:55:4 - ETA: 1:53:5 - ETA: 1:52:1 - ETA: 1:50:3 - ETA: 1:48:4 - ETA: 1:47:0 - ETA: 1:45:2 - ETA: 1:43:4 - ETA: 1:41:5 - ETA: 1:40:1 - ETA: 1:38:4 - ETA: 1:36:5 - ETA: 1:35:2 - ETA: 1:33:4 - ETA: 1:32:0 - ETA: 1:30:2 - ETA: 1:28:4 - ETA: 1:27:1 - ETA: 1:25:4 - ETA: 1:24:0 - ETA: 1:22:3 - ETA: 1:21:0 - ETA: 1:19:3 - ETA: 1:18:0 - ETA: 1:16:2 - ETA: 1:14:4 - ETA: 1:13:0 - ETA: 1:11:2 - ETA: 1:09:4 - ETA: 1:08:0 - ETA: 1:06:2 - ETA: 1:04:4 - ETA: 1:02:5 - ETA: 1:01:1 - ETA: 59:38  - ETA: 57:5 - ETA: 56:2 - ETA: 54:4 - ETA: 53:0 - ETA: 51:2 - ETA: 49:5 - ETA: 48:1 - ETA: 46:3 - ETA: 45:0 - ETA: 43:2 - ETA: 41:5 - ETA: 40:1 - ETA: 38:4 - ETA: 37:0 - ETA